In [ ]:
using Revise

In [ ]:
push!(LOAD_PATH, "/home/amir/work/mps/src/")

using QuantumModels
using ExactDiagonalizationTools
using MatrixProductStateTools
using TensorNetAlgs

using FFTW
using Plots
using LaTeXStrings

In [ ]:
lx = 16
maxdim = 40
mps = MPS{Float64}(lx, 2, noise=0.2);
center_at!(mps, 1)
mpo = xxz_mpo(Float64, lx, 2, 1.0)
env = initialenv(mps, mpo)

for sweep = 1:4
    println("Starting sweep number $sweep")
    dmrg2sitesweep!(mps, mpo, env, maxdim=maxdim, verbose=true)
    #println()
end

In [ ]:
println("Energy : ", measure(mps, mpo)[1]/(lx))
sz, sp, sm = spinoperators(1/2)
zzdata = measure(mps, sz, sz);
pmdata = measure(mps, sp, sm);

plt = plot(
    xaxis=(L"x", collect(2:2:lx)), 
    yaxis=(L"$\langle S_1 S_x\rangle$",),
    legend=:bottomright)
plot!(plt, 2:lx, [zzdata[(1, l)] for l in 2:lx], label=L"zz")
plot!(plt, 2:lx, [pmdata[(1, l)] for l in 2:lx], label=L"zz")

In [ ]:
ftdata = rfft(vcat([0], [zzdata[l, l+1] for l in 1:lx-1]))

plt = plot(xaxis=(L"q", (0, pi), ([0,pi/2,pi],[L"0",L"\pi/2", L"\pi"])))
plot!(plt, range(0,pi,length=div(lx,2)), real.(ftdata)[2:end])
plot!(plt, range(0,pi,length=div(lx,2)), imag.(ftdata)[2:end])